## Plan
bin irradiation levels and calculate std_deviation of dc_power per irratdiation grouping
model to predict the expected output (dc_power) given input features: 
- compare actual result with predicted result:
    - if actual result falls below 2 std_dev of irradiation grouping: flag as faulty
    - compare with other pannel readings at the same time (if same then cancel flagging)

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

#p1_url = 'https://github.com/LucMaingard/solar-plant/blob/main/data/raw/Plant_1_Generation_Data.csv?raw=true'
#p2_url = 'https://github.com/LucMaingard/solar-plant/blob/main/data/raw/Plant_2_Generation_Data.csv?raw=true'
#w1_url = 'https://github.com/LucMaingard/solar-plant/blob/main/data/raw/Plant_1_Weather_Sensor_Data.csv?raw=true'
#w2_url = 'https://github.com/LucMaingard/solar-plant/blob/main/data/raw/Plant_2_Weather_Sensor_Data.csv?raw=true'
p1_url = '/Users/lucmaingard/Dropbox/work/projects/solar/data/raw/Plant_1_Generation_Data.csv'
p2_url = '/Users/lucmaingard/Dropbox/work/projects/solar/data/raw/Plant_2_Generation_Data.csv'
w1_url = '/Users/lucmaingard/Dropbox/work/projects/solar/data/raw/Plant_1_Weather_Sensor_Data.csv'
w2_url = '/Users/lucmaingard/Dropbox/work/projects/solar/data/raw/Plant_2_Weather_Sensor_Data.csv'

# plant datasets
df_p1 = pd.read_csv(p1_url)
df_p2 = pd.read_csv(p2_url)

# weather datasets
df_w1 = pd.read_csv(w1_url)
df_w2 = pd.read_csv(w2_url)

In [2]:
#lowercase all column names
df_p1.columns=df_p1.columns.str.lower()
df_p2.columns=df_p2.columns.str.lower()
df_w1.columns=df_w1.columns.str.lower()
df_w2.columns=df_w2.columns.str.lower()
df_p1.columns

Index(['date_time', 'plant_id', 'source_key', 'dc_power', 'ac_power',
       'daily_yield', 'total_yield'],
      dtype='object')

In [3]:
#convert date_time to date time features
df_p1['time'] = pd.to_datetime(df_p1.date_time).dt.time
df_p1['date'] = pd.to_datetime(df_p1.date_time).dt.date
df_p1['week'] = pd.to_datetime(df_p1.date_time).dt.week
df_p1['hour'] = pd.to_datetime(df_p1.date_time).dt.hour
df_p1['minute'] = pd.to_datetime(df_p1.date_time).dt.minute

df_w1['time'] = pd.to_datetime(df_w1.date_time).dt.time
df_w1['date'] = pd.to_datetime(df_w1.date_time).dt.date

/var/folders/jw/n6m8n_yd6mx4qqq88m7zptsh0000gn/T/ipykernel_18002/2282812233.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_p1['week'] = pd.to_datetime(df_p1.date_time).dt.week


In [4]:
df_p1.date_time = pd.to_datetime(df_p1.date_time, format='%d-%m-%Y %H:%M')
df_w1.date_time = pd.to_datetime(df_w1.date_time, format='%Y-%m-%d %H:%M')

In [5]:
#left join df's 
df_solar = pd.merge(df_p1.drop(columns = ['plant_id']), df_w1.drop(columns = ['plant_id', 'source_key']), on=('date', 'time'), how='left')
df_solar.head()

,date_time_x,source_key,dc_power,ac_power,daily_yield,total_yield,time,date,week,hour,minute,date_time_y,ambient_temperature,module_temperature,irradiation
0,2020-05-15,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0
1,2020-05-15,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0
2,2020-05-15,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0
3,2020-05-15,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0
4,2020-05-15,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0


In [6]:
df_solar['dc_irradiation'] = df_solar['dc_power']/(df_solar['irradiation']+0.00000001)

In [7]:
df_solar.head()

,date_time_x,source_key,dc_power,ac_power,daily_yield,total_yield,time,date,week,hour,minute,date_time_y,ambient_temperature,module_temperature,irradiation,dc_irradiation
0,2020-05-15,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0,0.0
1,2020-05-15,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0,0.0
2,2020-05-15,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0,0.0
3,2020-05-15,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0,0.0
4,2020-05-15,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,00:00:00,2020-05-15,20,0,0,2020-05-15,25.184316,22.857507,0.0,0.0


In [8]:
from xgboost import XGBRegressor 

xgb = XGBRegressor()

/Users/lucmaingard/opt/miniconda3/envs/fast_api/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
cols = [
    'hour',
    'minute',
    'ambient_temperature',
    'module_temperature',
    'irradiation'
]
target = ['dc_power']

df = df_solar[target+cols].copy()

In [10]:
#df.shape

In [11]:
'''df=df.dropna()
df=df[df.dc_power>0]
df.shape'''

'df=df.dropna()\ndf=df[df.dc_power>0]\ndf.shape'

In [12]:
#df.iloc[:,1:].head()

In [13]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,1:]
y = df['dc_power']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((48144, 5), (10317, 5), (48144,), (10317,))

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

xgb.fit(X_train, y_train)

preds = xgb.predict(X_test)

mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
rmse = math.sqrt(mse)

mae, mse, rmse

/Users/lucmaingard/opt/miniconda3/envs/fast_api/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


(346.1398777905292, 720061.9917136995, 848.5646656052204)

In [16]:
import sys
import time
# function to train xgb model over n train_test_splits
def train_n_fols(model, X, y, n):

    start_time = time.time()

    mae = 0
    mse = 0
    rmse = 0

    for i in range(n):

        X = df.iloc[:,1:]
        y = df['dc_power']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=i)

        model.fit(X_train, y_train)

        preds = model.predict(X_test)

        mae += mean_absolute_error(y_test, preds)
        mse += mean_squared_error(y_test, preds)
        rmse += math.sqrt(mse)
        print(f"trained {i} folds", end='\r')
    
    mae=mae/n
    mse=mse/n
    rmse=rmse/n

    print("--- %s seconds total training time ---" % (time.time() - start_time))

    return mae, mse, rmse


In [17]:
#mae, mse, rmse = train_n_fols(xgb, X, y, n=50)

In [18]:
#mae, mse, rmse

In [19]:
#df.describe()

In [20]:
import statistics
statistics.stdev(y_test), statistics.stdev(pd.Series(preds))

(4024.3387680094415, 3927.6293934827318)

In [21]:
one_stdev_df_dc_irr = statistics.stdev(df_solar.dc_irradiation.dropna())
two_stdev_df_dc_irr = one_stdev_df_dc_irr*2
three_stdev_df_dc_irr = one_stdev_df_dc_irr*3

data_list = [one_stdev_df_dc_irr,two_stdev_df_dc_irr,three_stdev_df_dc_irr]
column_list = ['one_stdev_df_dc_irr','two_stdev_df_dc_irr','three_stdev_df_dc_irr']

In [22]:
#create processed stats csv 
# (stdevs of one_stdev_df_dc_irr) + add any relavent data (medians/means ofor imputation)

# create data frame 
def create_df(data_list, column_list):
    df = pd.DataFrame([data_list], columns=column_list)

    return df


In [23]:
import os
from datetime import datetime

path = "/Users/lucmaingard/Dropbox/work/projects/solar/data/processed_stats"

# write dataframe to processed_stats dir in data dir
def write_df(df,path):

    string_dt = datetime.now().strftime("%m.%d.%Y%H:%M:%S")
    
    if not os.path.exists(path):
      
        # if the demo_folder directory is not present 
        # then create it.
        os.makedirs(path)

    df.to_csv(path+"/"+string_dt+".csv")

    return
    

In [24]:
string_dt = datetime.now().strftime("%m.%d.%Y%H:%M:%S")
#datetime.strptime(string_dt, "%m/%d/%Y%H:%M:%S")
string_dt

'12.01.202210:55:14'

In [25]:
'''processed_stats_df = create_df(data_list, column_list)
#write_df()
processed_stats_df.head()'''

'processed_stats_df = create_df(data_list, column_list)\n#write_df()\nprocessed_stats_df.head()'

In [26]:
'''plt.scatter(df_solar.dc_power[df_solar.dc_power>0], df_solar.irradiation[df_solar.dc_power>0])
plt.show()'''

'plt.scatter(df_solar.dc_power[df_solar.dc_power>0], df_solar.irradiation[df_solar.dc_power>0])\nplt.show()'

In [27]:
df_solar.irradiation[df_solar.dc_power>0].describe()

count    24620.000000
mean         0.438595
std          0.292006
min          0.001575
25%          0.165297
50%          0.430854
75%          0.662152
max          1.221652
Name: irradiation, dtype: float64

In [28]:
'''
# creates bins to bin irradiation readings
def create_irr_bins():
    bin = 0.001
    add = 0.001
    bins = []

    for i in range(0,10):

        bins.append(round(bin, 4))
        bin+=add

    for i in range(0,10):
        bins.append(round(bin, 4))
        bin+=(add*10)

    while bin<=1.3:
        bins.append(round(bin, 4))
        bin+=(add*100)
    
    return bins'''




'\n# creates bins to bin irradiation readings\ndef create_irr_bins():\n    bin = 0.001\n    add = 0.001\n    bins = []\n\n    for i in range(0,10):\n\n        bins.append(round(bin, 4))\n        bin+=add\n\n    for i in range(0,10):\n        bins.append(round(bin, 4))\n        bin+=(add*10)\n\n    while bin<=1.3:\n        bins.append(round(bin, 4))\n        bin+=(add*100)\n    \n    return bins'

In [29]:
'''bins = create_irr_bins()'''

'bins = create_irr_bins()'

In [30]:
#bins = [0.001, 0.002, 0.003, 100]
#labels = ['C', 'B', 'A']
'''df_solar['test'] = pd.cut(x=df_solar['irradiation'], bins=bins)

df_solar[df_solar.dc_power>300].head()'''

"df_solar['test'] = pd.cut(x=df_solar['irradiation'], bins=bins)\n\ndf_solar[df_solar.dc_power>300].head()"

In [31]:
'''irr_groups_stdev = df_solar.groupby(by='test')['dc_power'].std()*2
write_df(irr_groups_stdev,path)'''

"irr_groups_stdev = df_solar.groupby(by='test')['dc_power'].std()*2\nwrite_df(irr_groups_stdev,path)"

In [32]:
###y.iloc[0]

In [33]:
#X_val.iloc[1:2]

In [87]:
X_val.iloc[800:801]

,hour,minute,ambient_temperature,module_temperature,irradiation
67464,9,0,24.688424,34.112601,0.338133


In [88]:
xgb.predict(X_val.iloc[800:801])[0]

4953.241

In [35]:
'''def predict(inference_object, model):
    
    prediction = model.predict(inference_object)

    return prediction'''

'def predict(inference_object, model):\n    \n    prediction = model.predict(inference_object)\n\n    return prediction'

In [36]:
import pandas as pd

In [37]:
test = {
    'hour':11,
    'minute':30,
    'ambient_temperature':24.4,
    'module_temperature':22.4,
    'irradiation':0.3,
    'dc_power':500
}

X2 = pd.json_normalize(test)
X = X2.iloc[:,:-1]
y = X2['dc_power']

In [38]:
#X2.iloc[:,:-1] , X2.iloc[:,-1], test['dc_power']

In [39]:
#test = pd.json_normalize(X2)

'''prediction = xgb.predict(X)[0]
prediction'''


'prediction = xgb.predict(X)[0]\nprediction'

In [40]:
'''prediction[0], y
if (float(y-prediction) > float(mae*2)):
    print('maintenance needed')
else:
    print('panel is performing well')'''

"prediction[0], y\nif (float(y-prediction) > float(mae*2)):\n    print('maintenance needed')\nelse:\n    print('panel is performing well')"

In [41]:
from fastapi import FastAPI

# Initialize FastAPI app
app = FastAPI()

In [42]:
'''pd.json_normalize(test)['dc_power']'''

"pd.json_normalize(test)['dc_power']"

In [43]:
def get_model_response(input_object, model, mae):

    # converts json into df
    X2 = pd.json_normalize(input_object)
    X = X2.iloc[:,:-1]
    true_dc_power = X2['dc_power']
    prediction = model.predict(X)[0]
    
    # if the true value is below 2 stdeviations of the expected value, flag as 
    if float(true_dc_power-prediction) >= float(mae*2):
        label = "Maintenance required"
    else:
        label = "Panel is performing well"

    '''return {
        'label': str(label),
        'expected_dc_power_gen': prediction,
        'actual_dc_power_gen': true_dc_power
    }'''
    return prediction

In [70]:
import datetime 
str(datetime.datetime.now())

'2022-12-01 12:02:00.756883'

In [76]:
MODEL_DIR_PATH = '/Users/lucmaingard/Dropbox/work/projects/solar/models/'
# Save as JSON file
if not os.path.exists(MODEL_DIR_PATH):
      
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs(MODEL_DIR_PATH)

current_timestamp = str(datetime.datetime.now()).replace(" ","")
VERSIONING_PATH = MODEL_DIR_PATH+current_timestamp+'/'

if not os.path.exists(VERSIONING_PATH):
      
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs(VERSIONING_PATH)


xgb.save_model(MODEL_DIR_PATH+"model.json")
xgb.save_model(VERSIONING_PATH+"model.json")


In [91]:
PATH = '/Users/lucmaingard/Dropbox/work/projects/solar/data/processed_stats/'

# Save as JSON file
if not os.path.exists(PATH):
    
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs(PATH)

df = pd.DataFrame(columns=['mae', 'mse', 'rmse'], data=[[mae, mse, rmse]])
df.to_csv(PATH+'best_model_scores.csv')

In [93]:
# read scores for new model train analysis
rmse = pd.read_csv('/Users/lucmaingard/Dropbox/work/projects/solar/data/processed_stats/best_model_scores.csv')['rmse'].values
rmse

array([848.56466561])

In [56]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()
cwd

'/Users/lucmaingard/Dropbox/work/projects/solar/notebooks'

In [ ]:
xbg_reg = XGBRegressor()

xbg_reg.load_model("/Users/lucmaingard/Dropbox/work/projects/solar/models/model.json")

In [94]:
model_path = '/Users/lucmaingard/Dropbox/work/projects/solar/models/model.json'

# load trained xgb model
xgb = XGBRegressor()
xgb.load_model(model_path)

In [95]:
model

In [45]:
# Third party imports
from pydantic import BaseModel
from fastapi import Request

# Input for data validation
class Input(BaseModel):
    hour: int
    minute: int
    ambient_temperature: float
    module_temperature: float
    irradiation: float
    dc_power: int 



@app.get('/info')
async def model_info():
    """Return model information, version, how to call"""
    return {
        "name": 'DC Power Generation Predictor',
        "version": 'v1'
    }

@app.post("/items")
async def create_item(request: Request):

    test = pd.json_normalize(request.json())
    
    return test['ambient_temperature']


@app.post("/items/")
async def create_item(request: Request):
    
    return request['ambient_temperature']


@app.get('/health')
async def service_health():
    """Return service health"""
    return {
        "ok"
    }

@app.get('/train')
async def service_health():
    """Trigger model retraining"""
    return {
        "ok"
    }

@app.get('/{x1}/{x2}/{x3}/{x4}/{x5}/predict')
async def model_predict(x1:int, x2:int, x3:float, x4:float, x5:float):
    """Predict with input"""
    l1 = [x1, x2, x3, x4,x5]

    x = pd.DataFrame(columns=['hour', 'minute', 'ambient_temperature', 'module_temperature', 'irradiation'],data=[l1])

    pred = xgb.predict(x)
    dict_out = {}
    for count, value in enumerate(pred):
        dict_out['expected_dc_power'] = float(value)
    '''input_object = input.dict()
    response = get_model_response(input_object, xgb, mae)'''
    return dict_out




In [46]:
import json 

l1 = [11, 30, 24.4, 22.4,0.3]
x = pd.DataFrame(columns=['hour', 'minute', 'ambient_temperature', 'module_temperature', 'irradiation'],data=[l1])
predic = xgb.predict(x)
print(predic)
dict_out = {}
for count, value in enumerate(predic):
    dict_out['count'] = float(value)


dict_out


[5377.5244]


{'count': 5377.5244140625}

In [ ]:
test

{'hour': 11,
 'minute': 30,
 'ambient_temperature': 24.4,
 'module_temperature': 22.4,
 'irradiation': 0.3,
 'dc_power': 500}

In [47]:
response = get_model_response(test, xgb, mae)
response

5377.5244

In [48]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)

ngrok_tunnel

<NgrokTunnel: "http://42c8-169-0-95-131.ngrok.io" -> "http://localhost:8000">

In [49]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [18002]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51536 - "GET /11/30/24.2/22.2/0.3/predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18002]


In [ ]:
test

{'hour': 11,
 'minute': 30,
 'ambient_temperature': 24.4,
 'module_temperature': 22.4,
 'irradiation': 0.3,
 'dc_power': 500}